# Goal

The goal of this notebook is to help in interactively developing the new multi-objective version of the Tomograph.

## Goals for the Tomograph

1. Move away from matplotlib to either plotly, or bokeh. 
2. Make sure that:
    1. All scales are the same so that they are comparable.
    2. We should be able to plot not just predictions but also individual observations as a scatterplot. 
    3. We should be able to link the points across multiple charts to investigate them more deeply (bokeh seems to be the winner here)
    4. We should be able to connect multiple tomographs to the same optimizer, as any given analysis might focus on different aspects and each aspect should have it's own graphs.
3. The Tomograph produces a set of heatmaps that belong to 2D cross-sections of the higher-dimensional hypercube. All these cross-sections share a single point (often the optimum). The user should be able to:
    1. Select one of the predefined points (different types of optima, maybe optima for different contexts)
    2. View the resulting cross sections and sensitivity analysis for such a point
    3. Be able to use sliders to adjust the point so that they can easily "walk the hypercube"
    

Plotly has the advantage of also enabling 3D visualizations of the pareto frontier and generates surface plots etc. But bokeh has built-in support for linking data. 

Given what we need, I would start with bokeh, for dabl-style and tomograph plots, and then switch to plotly for parameter interactions, and 3D pareto visualizations.

https://docs.bokeh.org/en/latest/index.html

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory, objective_function_config_store
objective_function_config_store.list_named_configs()

[Name: default
 
 Description: None
 
 Config Values: {
   "implementation": "Hypersphere",
   "hypersphere_config.num_objectives": 3,
   "hypersphere_config.minimize": "all",
   "hypersphere_config.radius": 10,
   "description": "default"
 },
 Name: three_level_quadratic
 
 Description: None
 
 Config Values: {
   "implementation": "ThreeLevelQuadratic"
 },
 Name: flower
 
 Description: None
 
 Config Values: {
   "implementation": "Flower"
 },
 Name: noisy_polynomial_objective
 
 Description: None
 
 Config Values: {
   "implementation": "PolynomialObjective",
   "polynomial_objective_config.seed": 17,
   "polynomial_objective_config.input_domain_dimension": 2,
   "polynomial_objective_config.input_domain_min": -1024,
   "polynomial_objective_config.input_domain_width": 2048,
   "polynomial_objective_config.max_degree": 2,
   "polynomial_objective_config.include_mixed_coefficients": true,
   "polynomial_objective_config.percent_coefficients_zeroed": 0,
   "polynomial_objective_config

In [3]:
#objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_2_mutually_exclusive_polynomials")
#objective_function_config = objective_function_config_store.get_config_by_name("three_level_quadratic")
objective_function_config = objective_function_config_store.get_config_by_name("2d_hypersphere_minimize_none")
objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_waves_3_params_2_objectives_half_pi_phase_difference")

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)
optimization_problem = objective_function.default_optimization_problem

In [4]:
print(objective_function.default_optimization_problem)

Parameter Space:
  Name: domain
  Dimensions:
    x_0: [0.00, 6.28]
    x_1: [0.00, 6.28]
    x_2: [0.00, 6.28]
------------------------------------------------------------
Context Space:
None
------------------------------------------------------------
Feature Space:
  Name: features
  Dimensions:
    contains_context: {False}

  IF contains_context IN {False} THEN (
    Name: domain
    Dimensions:
      x_0: [0.00, 6.28]
      x_1: [0.00, 6.28]
      x_2: [0.00, 6.28]
  )
------------------------------------------------------------
Objective Space:
  Name: range
  Dimensions:
    y0: [-inf, inf]
    y1: [-inf, inf]
------------------------------------------------------------
Objectives:
[{'name': 'y0', 'minimize': True}, {'name': 'y1', 'minimize': True}]


In [5]:
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
optimizer_factory = BayesianOptimizerFactory()
optimizer_config = bayesian_optimizer_config_store.default
optimizer = optimizer_factory.create_local_optimizer(
    optimizer_config=optimizer_config,
    optimization_problem=optimization_problem
)

03/12/2021 18:13:11 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  40 -    create_local_optimizer() ] Creating a bayesian optimizer with config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.7,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_m

03/12/2021 18:13:11 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_2_input_space
  Dimensions:
    domain___x_1: [0.00, 6.28]
    domain___x_2: [0.00, 6.28]
    domain___x_0: [0.00, 6.28]
    contains_context: {False}
03/12/2021 18:13:11 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_3_input_space
  Dimensions:
    domain___x_0: [0.00, 6.28]
    domain___x_1: [0.00, 6.28]
    domain___x_2: [0.00, 6.28]
    contains_context: {False}
03/12/2021 18:13:11 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_4_input_space
  Dimensions:
    domain___x_2

In [6]:
params_df = objective_function.parameter_space.random_dataframe(num_samples=2000)
objectives_df = objective_function.evaluate_dataframe(params_df)

In [7]:
optimizer.register(parameter_values_pandas_frame=params_df, target_values_pandas_frame=objectives_df)

03/12/2021 18:13:19 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 153 -                  register() ] Registering 2000 parameters and 2000 objectives.


In [8]:
from mlos.Spaces import Point
from mlos.OptimizerMonitoring.Tomograph.ModelTomograph import ModelTomograph

tomograph = ModelTomograph(optimizer=optimizer)

In [ ]:
config = objective_function.parameter_space.random()

In [ ]:
import os
os.getpid()

In [ ]:
tomograph.plot()

## Plotting Observations
The goal of this next section is to plot all observations on a grid where:
   1. On the diagonal we plot a single parameters vs. objective value (basically what dabl does already)
   2. On the other subgraphs we plot two parameters colored according to the objective function value. Additionally, we can also regulate their opacity as a function of their distance from the plane of the graph.

In [23]:
from typing import List

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import CategoricalTicker, ColorBar, ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.plotting import figure
from bokeh.transform import factor_mark

import pandas as pd

from mlos.Logger import create_logger
from mlos.Optimizers.OptimizationProblem import OptimizationProblem
from mlos.Optimizers.OptimizerBase import OptimizerBase
from mlos.Spaces import CategoricalDimension, Point
from mlos.Spaces.HypergridAdapters import CategoricalToDiscreteHypergridAdapter
from mlos.Utils.KeyOrderedDict import KeyOrderedDict


class ObservationsDataSource:
    """Maintains data source that the individual GridPlots can use.
    """
    def __init__(self, optimization_problem: OptimizationProblem, parameters_df: pd.DataFrame, context_df: pd.DataFrame, objectives_df: pd.DataFrame, pareto_df: pd.DataFrame):
        self.optimization_problem = optimization_problem
        self._feature_space_adapter = CategoricalToDiscreteHypergridAdapter(adaptee=self.optimization_problem.feature_space)
        
        self.parameters_df: pd.DataFrame = None
        self.context_df: pd.DataFrame = None
        self.objectives_df: pd.DataFrame = None
        self.pareto_df: pd.DataFrame = None
        self.observations_df: pd.DataFrame = None
        
        self.data_source: ColumnDataSource = None
        self.pareto_data_source: ColumnDataSource = None
        self.dominated_data_source: ColumnDataSource = None
        
        
        self.data_source = ColumnDataSource()
        self.pareto_data_source = ColumnDataSource()
        self.dominated_data_source = ColumnDataSource()
        
        self.update_data(parameters_df=parameters_df, context_df=context_df, objectives_df=objectives_df, pareto_df=pareto_df)
        
    def update_data(self, parameters_df: pd.DataFrame, context_df: pd.DataFrame, objectives_df: pd.DataFrame, pareto_df: pd.DataFrame):
        self.parameters_df = parameters_df
        self.context_df = context_df
        self.objectives_df = objectives_df
        self.pareto_df = pareto_df
        self.observations_df = self._construct_observations()
        
        # In order to preserve the identity of the data sources, we create temporary ones, and then copy their data over to the data sources in use by the grid plot.
        #
        temp_data_source = ColumnDataSource(data=self.observations_df)
        temp_pareto_data_source = ColumnDataSource(data=self.observations_df[self.observations_df['is_pareto']])
        temp_dominated_data_source = ColumnDataSource(data=self.observations_df[~self.observations_df['is_pareto']])
        
        self.data_source.data = dict(temp_data_source.data)
        self.pareto_data_source.data = dict(temp_pareto_data_source.data)
        self.dominated_data_source.data = dict(temp_dominated_data_source.data)
        
    def _construct_observations(self):
        features_df = self.optimization_problem.construct_feature_dataframe(parameters_df=self.parameters_df, context_df=self.context_df, product=False)
        projected_features_df = self._feature_space_adapter.project_dataframe(features_df)
        observations_df = pd.concat([projected_features_df, objectives_df], axis=1)
        observations_df['is_pareto'] = False
        observations_df.loc[pareto_df.index, 'is_pareto'] = True
        return observations_df
        

class GridPlot:
    """Maintains all data, meta-data and styling information required to produce a grid-plot.

    The grid plot is built based on the OptimizationProblem instance, to find out what objectives and what
    features are to be plotted. We use information contained in the dimensions to compute the ranges for all
    axes/ranges on the plot, as well as to configure the color map.

    If the range is infinite (as can be the case with many objectives) we can use the observed range of values to
    configure the range of values to be plotted.

    Each figure in the grid plot contains:
    * Either a scatter plot of feature vs. feature where the color of each point corresponds to the value of the objective value
    * Or a scatter plot of feature vs. objective (if we are on a diagonal).

    Additionally, as an extension, we could also plot the predicted values as a background heatmap for the feature vs. feature
    plots, and a predicted value with confident intervals plot for feature vs. objective plots. This of course introduces a complication
    of needing to query the optimizer for each pixel and so we will add it later.
    """

    def __init__(
        self,
        optimization_problem: OptimizationProblem,
        objective_name: str,
        observations_data_source: ObservationsDataSource,
        logger=None
    ):
        if logger is None:
            logger = create_logger(self.__class__.__name__)
        self.logger = logger
        
        
        # Let's start with the data section. This will be populated by the Tomograph.
        #
        self._observations_data_source = observations_data_source
                
        #  Metatdata - what dimensions are we going to be plotting here?
        #
        self.optimization_problem = optimization_problem
        assert objective_name in self.optimization_problem.objective_names
        self.objective_name = objective_name

        
        self._feature_space_adapter = CategoricalToDiscreteHypergridAdapter(adaptee=self.optimization_problem.feature_space)

        self.feature_dimension_names: List[str] = [
            feature_name
            for feature_name
            in self._feature_space_adapter.dimension_names
            if feature_name != "contains_context"
        ]
        self.num_features = len(self.feature_dimension_names)

        # Finally, below is all the data and objects that bokeh needs to render this grid plot.
        #
            
        
        # Plot options including tooltips.
        #
        

        # Stores figure ranges by name so that we can synchronize zooming and panning
        #
        self._x_ranges_by_name = {}
        self._y_ranges_by_name = {}

        # Stores an array of all plots for all objectives.
        #
        self._figures = [
            [None for col in range(self.num_features)]
            for row in range(self.num_features)
        ]
        

        # Stores the bokeh gridplot object.
        #
        self._grid_plot = None
        
        
    @property
    def grid_plot(self):
        return self._grid_plot

        
    def update_plots(self):
        """Updates the plot with new observations.
        """
                
        self._x_ranges_by_name = {}
        self._y_ranges_by_name = {}
        self._grid_plot = None
        
        
        tooltips = [(f"{feature_name}", f"@{feature_name}") for feature_name in self.feature_dimension_names]
        tooltips.extend([(f"{objective_name}", f"@{objective_name}") for objective_name in self.optimization_problem.objective_names])
        hover = HoverTool(tooltips=tooltips)
        
        plot_options = dict(
            plot_width=int(2000 / self.num_features),
            plot_height=int(2000 / self.num_features),
            tools=['box_select', 'lasso_select', 'box_zoom', 'wheel_zoom', 'reset', hover]
        )
        
        final_column_plot_options = dict(
            plot_width=int(2000 / self.num_features) + 75,
            plot_height=int(2000 / self.num_features),
            tools=['box_select', 'lasso_select', 'box_zoom', 'wheel_zoom', 'reset', hover]
        )
        
        color_mapper = LinearColorMapper(palette='Turbo256', low=self._observations_data_source.observations_df[self.objective_name].min(), high=self._observations_data_source.observations_df[self.objective_name].max())

        for row, row_dimension_name in enumerate(self.feature_dimension_names):
            for col, col_dimension_name in enumerate(self.feature_dimension_names):

                x_axis_name = col_dimension_name
                x_ticks, x_tick_label_mapping = self._get_feature_ticks_and_tick_label_mapping(x_axis_name)

                if row == col:
                    # For plots on the diagonals, we want to plot the row dimension vs. objective
                    #
                    y_axis_name = self.objective_name
                    y_ticks, y_tick_label_mapping = None, None
                else:
                    y_axis_name = row_dimension_name
                    y_ticks, y_tick_label_mapping = self._get_feature_ticks_and_tick_label_mapping(y_axis_name)
                    
                
                if col == (self.num_features - 1):
                    fig = figure(**final_column_plot_options)
                else:
                    fig = figure(**plot_options)

                fig.scatter(
                    x_axis_name,
                    y_axis_name,
                    color={'field': self.objective_name, 'transform': color_mapper},
                    marker='circle',
                    source=self._observations_data_source.dominated_data_source,
                    legend_label="dominated",
                    muted_alpha=0.02 # TODO: figure out how to have clicking on the legend mute unselected points.
                )
                
                fig.scatter(
                    x_axis_name,
                    y_axis_name,
                    color={'field': self.objective_name, 'transform': color_mapper},
                    marker='triangle',
                    source=self._observations_data_source.pareto_data_source,
                    legend_label="pareto optimal",
                    muted_alpha=0.02 # TODO: figure out how to have clicking on the legend mute unselected points.
                )
                
                fig.legend.click_policy="hide"
                fig.xaxis.axis_label = x_axis_name
                fig.yaxis.axis_label = y_axis_name


                fig.xaxis.ticker = x_ticks
                fig.axis.major_label_overrides = x_tick_label_mapping

                if y_ticks is not None:
                    fig.yaxis.ticker = y_ticks
                    fig.yaxis.major_label_overrides = y_tick_label_mapping

                self._set_ranges(fig, x_axis_name, y_axis_name)

                self.logger.debug(f"Assigning figure to [{row}][{col}]. {self.objective_name}, {row_dimension_name}, {col_dimension_name}")
                self._figures[row][col] = fig
            
            
            color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title=self.objective_name)
            self._figures[row][-1].add_layout(color_bar, 'right')


        self._grid_plot = gridplot(self._figures)
        
        
    def _get_feature_ticks_and_tick_label_mapping(self, axis_name):
        projected_ticks = self._feature_space_adapter[axis_name].linspace(5)
        projected_ticks_df = pd.DataFrame({axis_name: projected_ticks})
        unprojected_ticks_df = self._feature_space_adapter.unproject_dataframe(projected_ticks_df)
        unprojected_col_name = unprojected_ticks_df.columns[0] 
        tick_mapping = {
            projected_tick: f"{unprojected_tick:.2f}" if isinstance(unprojected_tick, float) else str(unprojected_tick)
            for projected_tick, unprojected_tick
            in zip(projected_ticks, unprojected_ticks_df[unprojected_col_name])
        }
        return projected_ticks, tick_mapping
    

    def _set_ranges(self, fig, x_axis_name, y_axis_name):
        """Sets the ranges on each axis to enable synchronized panning and zooming."""
        x_range = self._x_ranges_by_name.get(x_axis_name, None)
        if x_axis_name in self._x_ranges_by_name:
            fig.x_range = self._x_ranges_by_name[x_axis_name]
        else:
            self._x_ranges_by_name[x_axis_name] = fig.x_range
        
        if y_axis_name in self._y_ranges_by_name:
            fig.y_range = self._y_ranges_by_name[y_axis_name]
        else:
            self._y_ranges_by_name[y_axis_name] = fig.y_range

            
            
            
            
            
class Tomograph2:
    """"""    
    
    
    def __init__(self, optimizer: OptimizerBase, logger=None):
        if logger is None:
            logger = create_logger(self.__class__.__name__)
        self.logger = logger
        
        self.optimizer = optimizer        
        self.optimization_problem = optimizer.optimization_problem
        
        params_df, objectives_df, context_df = self.optimizer.get_all_observations()
        pareto_df = self.optimizer.pareto_frontier.pareto_df
        
        self.observations_data_source = ObservationsDataSource(
            optimization_problem=self.optimization_problem,
            parameters_df=params_df,
            context_df=context_df,
            objectives_df=objectives_df,
            pareto_df=pareto_df
        )

        self.grid_plots_by_objective_name = KeyOrderedDict(ordered_keys=self.optimization_problem.objective_names, value_type=GridPlot)
        
        for objective_name in self.optimization_problem.objective_names:
            self.grid_plots_by_objective_name[objective_name] = GridPlot(
                optimization_problem=self.optimization_problem,
                objective_name=objective_name,
                observations_data_source=self.observations_data_source,
                logger=self.logger
            )
        
        self._feature_space_adapter = CategoricalToDiscreteHypergridAdapter(adaptee=self.optimization_problem.feature_space)        
        
    
    def plot_observations(self, objective_names: List[str] = None, refresh_data: bool = True):
        """Plot all observations.
        """
            
        self.logger.debug(f"Plotting observations")
        
        if objective_names is None:
            objective_names = [self.optimization_problem.objective_names[0]]
            
        if refresh_data:
            params_df, objectives_df, context_df = self.optimizer.get_all_observations()
            pareto_df = self.optimizer.pareto_frontier.pareto_df
            self.observations_data_source.update_data(parameters_df=params_df, context_df=context_df, objectives_df=objectives_df, pareto_df=pareto_df)
        
        for objective_name in objective_names:
            grid_plot = self.grid_plots_by_objective_name[objective_name]
            grid_plot.update_plots()
            show(grid_plot.grid_plot)
            
    
    
    
    
    
    
    
        
output_notebook()       
tomograph2 = Tomograph2(optimizer=optimizer)
tomograph2.plot_observations(objective_names=optimization_problem.objective_names)

Loading BokehJS ...

In [ ]:
params_df, objectives_df, context_df = optimizer.get_all_observations()
pareto_df = optimizer.pareto_frontier.pareto_df

In [18]:
params_df

,x_0,x_1,x_2
0,2.988934,0.524599,2.525177
1,0.822751,3.850707,2.462898
2,1.038170,0.257309,3.056428
3,1.887678,3.186271,1.008407
4,4.218109,0.052306,4.733373
...,...,...,...
1995,4.150409,0.178142,0.300180
1996,4.216042,4.399305,0.860154
1997,5.436225,1.086035,4.328451
1998,2.870729,2.467309,2.294967


In [17]:
     
observations_data_source = ObservationsDataSource(
    optimization_problem=optimization_problem,
    parameters_df=params_df,
    context_df=context_df,
    objectives_df=objectives_df,
    pareto_df=pareto_df
)

TypeError: update_data() got multiple values for argument 'parameters_df'

In [ ]:
tomograph2.plot_observations(objective_names=optimization_problem.objective_names)